In [2]:
## WDPA and WD-OECM pre-processing script - public and restricted data (Python)

# Import necessary Libraries
from dateutil.relativedelta import relativedelta
import datetime
from arcgis.gis import GIS
import arcpy
import math
import os
from arcgis.features import FeatureLayer
from arcgis.features import FeatureLayerCollection
import pandas as pd

#Itentify the month and year corresponding to the version of the WDPA and WD-OECM used in the Protected Planet Report 2024 (August 2024)
date = datetime.datetime(2024, 8, 1)
month = date.strftime("%b")  # "Aug"
Month = date.strftime("%B")   # "August"
year = date.strftime("%Y")    # "2024"

In [ ]:
#Set default file geodatabase for intermediate outputs
arcpy.env.workspace = r"path_to_file_geodatabase_for_intermediate_outputs"

#Define paths to WDPA and WDOECM points and polygons 

# Public WDPA paths
wdpa_path = r'path_to_all_public_wdpa_point_polygon_file_geodatabase'
wdpa_poly_path = os.path.join(wdpa_path,'WDPA_poly_{0}{1}'.format(month,year))
wdpa_point_path = os.path.join(wdpa_path,'WDPA_point_{0}{1}'.format(month,year))
print(wdpa_path)

# Public WDOECM paths
wdoecm_path = r'path_to_all_public_wdoecm_point_polygon_file_geodatabase'
wdoecm_poly_path = os.path.join(wdoecm_path,'WDOECM_poly_{0}{1}'.format(month,year)) 
wdoecm_point_path = os.path.join(wdoecm_path,'WDOECM_point_{0}{1}'.format(month,year))

#Restricted WDPA paths 
restricted_wdpa_path = r'path_to_all_restricted_wdpa_point_polygon_file_geodatabase'
restricted_wdpa_poly_path = os.path.join(restricted_wdpa_path,'wdpa_poly_restricted')
restricted_wdpa_point_path = os.path.join(restricted_wdpa_path,'wdpa_point_restricted')

Datasets = ["wdpa", "wdoecm"]

#Read in and subset WDPA and WDOECM

# Public WDPA points
## Export features where REP_AREA isn't equaly to "0", DESIG_ENG is not equal to "UNESCO-MAB Biosphere Reserve", and STATUS is not equal to either "Not Reported" or "Proposed"
arcpy.conversion.ExportFeatures(wdpa_point_path,r"memory\wdpa_point_mp", "REP_AREA <> 0 And DESIG_ENG <> 'UNESCO-MAB Biosphere Reserve' And STATUS <> 'Not Reported' And STATUS <> 'Proposed'")
## Convert to single part
arcpy.management.MultipartToSinglepart("wdpa_point_mp", "wdpa_point")

# Public WDPA polygons
## Export features where DESIG_ENG is not equal to "UNESCO-MAB Biosphere Reserve", and STATUS is not equal to either "Not Reported" or "Proposed"
arcpy.conversion.ExportFeatures(wdpa_poly_path,r"memory\wdpa_poly_mp", "DESIG_ENG <> 'UNESCO-MAB Biosphere Reserve' And STATUS <> 'Not Reported' And STATUS <> 'Proposed'")
## Convert to single part
arcpy.management.MultipartToSinglepart("wdpa_poly_mp", "wdpa_poly")

# Restricted WDPA points
## Export features where REP_AREA isn't equaly to "0", DESIG_ENG is not equal to "UNESCO-MAB Biosphere Reserve", and STATUS is not equal to either "Not Reported" or "Proposed"
arcpy.conversion.ExportFeatures(restricted_wdpa_point_path,r"memory\restricted_wdpa_point_mp", "REP_AREA <> 0 And DESIG_ENG <> 'UNESCO-MAB Biosphere Reserve' And STATUS <> 'Not Reported' And STATUS <> 'Proposed'")
## Convert to single part
arcpy.management.MultipartToSinglepart("restricted_wdpa_point_mp", "restricted_wdpa_point")

# Restricted WDPA polygons
## Export features where DESIG_ENG is not equal to "UNESCO-MAB Biosphere Reserve", and STATUS is not equal to either "Not Reported" or "Proposed"
arcpy.conversion.ExportFeatures(restricted_wdpa_poly_path,r"memory\restricted_wdpa_poly_mp", "DESIG_ENG <> 'UNESCO-MAB Biosphere Reserve' And STATUS <> 'Not Reported' And STATUS <> 'Proposed'")
## Convert to single part
arcpy.management.MultipartToSinglepart("restricted_wdpa_poly_mp", "restricted_wdpa_poly")

# Public WDOECM points
## Export features where REP_AREA is not equal to "0" and STATUS is not equal to either "Not Reported" or "Proposed"
arcpy.conversion.ExportFeatures(wdoecm_point_path,r"memory\wdoecm_point_mp", "")
## Convert to single part
arcpy.management.MultipartToSinglepart("wdoecm_point_mp", "wdoecm_point")

## Export features where STATUS is not equal to either "Not Reported" or "Proposed"
arcpy.conversion.ExportFeatures(wdoecm_poly_path,r"memory\wdoecm_poly_mp", "")
## Convert to single part
arcpy.management.MultipartToSinglepart("wdoecm_poly_mp", "wdoecm_poly")

##Buffer WDPA, WDOECM, and restricted WDPA point shapefiles to REP_AREA value; converting to polygon shapefiles. 
Datasets = ["wdpa", "wdoecm", "restricted_wdpa"]

for i in Datasets:
    point_fc = f"{i}_point"
    
    # Add buffer_radius field
    arcpy.management.AddField(point_fc, "buffer_radius", "LONG", field_length=None, field_alias="", field_is_nullable="NULLABLE", field_is_required="NON_REQUIRED")
    
    # Calculate the buffer radius based on the REP_AREA field value
    arcpy.management.CalculateField(point_fc, "buffer_radius", "math.sqrt(!REP_AREA!/3.1415)", "PYTHON3", code_block="")
    
    # Create a buffer field combining the radius with a unit string
    arcpy.management.CalculateField(point_fc, "buffer_field", 'str(!buffer_radius!) + " Kilometers"', "PYTHON3", code_block="")
    
    # Perform the buffering analysis
    buffer_fc = f"{i}_point_buffered"
    arcpy.analysis.PairwiseBuffer(point_fc, buffer_fc, "buffer_field", "None", None, "GEODESIC", "0 Meters")

## Repair geometries of buffered points layers.
arcpy.management.RepairGeometry("wdpa_point_buffered", "KEEP_NULL", "OGC")
arcpy.management.RepairGeometry("wdoecm_point_buffered", "KEEP_NULL", "OGC")
arcpy.management.RepairGeometry("restricted_wdpa_point_buffered", "KEEP_NULL", "OGC")

## Merge WDPA points + polys (all), WDOECM points + polys (all), and erase protected area extent from OECM extent to produce a
# layer show additional coverage provided by OECMs only. 
arcpy.management.Merge(["wdpa_point_buffered", "wdpa_poly", "restricted_wdpa_point_buffered", "restricted_wdpa_poly"], "wdpa_merged_public_restricted")
arcpy.management.Merge(["wdoecm_point_buffered", "wdoecm_poly"], "wdoecm_merged")
arcpy.analysis.PairwiseErase("wdoecm_merged", "wdpa_merged_public_restricted", "wdoecm_merged_pa_erase", "")

## Repair geometries of newly created layers
arcpy.management.RepairGeometry("wdpa_merged_public_restricted", "KEEP_NULL", "OGC")
arcpy.management.RepairGeometry("wdoecm_merged", "KEEP_NULL", "OGC")
arcpy.management.RepairGeometry("wdoecm_merged_pa_erase", "KEEP_NULL", "OGC")

# Write intermediate WDPA and WDOECM to local file geodatabases before clearing cache to free memory. 
#WDPA
##Set working environment for output files
wdpa_temp_path = r'path_to_file_geodatabase_for_intermediate_outputs'
wdpa_temp_poly_point_path = os.path.join(wdpa_temp_path,"wdpa_merged_public_restricted")
wdpa_output_path = r'path_to_file_geodatabase_for_preprocessing_outputs'
wdpa_output_poly_point_path = os.path.join(wdpa_output_path,"wdpa_poly_buffpoint_{0}{1}".format(month, year))
arcpy.conversion.ExportFeatures(wdpa_temp_poly_point_path, wdpa_output_poly_point_path)

#WDOECM
wdoecm_temp_path = r'path_to_file_geodatabase_for_intermediate_outputs'
wdoecm_temp_poly_point_path = os.path.join(wdoecm_temp_path,"wdoecm_merged")
wdoecm_output_path = r'path_to_file_geodatabase_for_preprocessing_outputs'
wdoecm_output_poly_point_path = os.path.join(wdoecm_output_path,"wdoecm_poly_buffpoint_{0}{1}".format(month, year))
arcpy.conversion.ExportFeatures(wdoecm_temp_poly_point_path, wdoecm_output_poly_point_path)

wdoecm_pa_erase_temp_path = r'path_to_file_geodatabase_for_intermediate_outputs'
wdoecm_pa_erase_temp_poly_point_path = os.path.join(wdoecm_pa_erase_temp_path,"wdoecm_merged_pa_erase")
wdoecm_pa_erase_output_path = r'path_to_file_geodatabase_for_preprocessing_outputs'
wdoecm_pa_erase_output_poly_point_path = os.path.join(wdoecm_pa_erase_output_path,"wdoecm_pa_erase_poly_buffpoint_{0}{1}".format(month, year))
arcpy.conversion.ExportFeatures(wdoecm_pa_erase_temp_poly_point_path, wdoecm_pa_erase_output_poly_point_path)

# Define the path to the geodatabase and the feature class
basemap_path = r'path_to_file_geodatabase_containing_basemap'
basemap_name = 'basemap_name'
basemap_path_name = os.path.join(basemap_path, basemap_name)
basemap = arcpy.management.MakeFeatureLayer(basemap_path_name, basemap_name)

#Pairwise intersect WDPA and WD-OECM in memory with basemap
##WDPA
#Intersect (pairwise) WDPA (polygon + point data) with global basemap
arcpy.analysis.PairwiseIntersect([wdpa_output_poly_point_path, basemap], "wdpa_poly_point_basemap_intersect", "ALL", "", "INPUT")
#Repair interesected WDPA
arcpy.management.RepairGeometry("wdpa_poly_point_basemap_intersect", "KEEP_NULL", "OGC")
#Export intersected WDPA from memory to output file geodatabase
arcpy.conversion.FeatureClassToFeatureClass("wdpa_poly_point_basemap_intersect", wdpa_output_path, "wdpa_poly_point_public_restricted_basemap_intersect_{0}{1}".format(month, year))

##WDOECM (verion with PA extent erased)
#Intersect (pairwise) WDOECM (polygon + point data) with global basemap
arcpy.analysis.PairwiseIntersect([wdoecm_pa_erase_output_poly_point_path, basemap], "wdoecm_basemap_intersect", "ALL", "", "INPUT")
#Repair intesected WDOECM
arcpy.management.RepairGeometry("wdoecm_basemap_intersect", "KEEP_NULL", "OGC")
#Export intersected WDOECM from memory to output file geodatabase
arcpy.conversion.FeatureClassToFeatureClass("wdoecm_basemap_intersect", wdoecm_output_path, "wdoecm_basemap_intersect_{0}{1}".format(month, year))

#Split WDPA and WDOECM by STATUS_YR field to assess change in coverage over time. 
wdpa_public_restricted_basemap_intersect_output_path = os.path.join(wdpa_output_path,"wdpa_poly_point_public_restricted_basemap_intersect_{0}{1}".format(month, year))

#Define output paths for WDPA and WDOECM subset by STATUS_YR
wdpa_status_yr_2020_and_less_path = os.path.join(wdpa_output_path,"wdpa_poly_point_public_restricted_basemap_intersect_statusyr_2020_and_less_{0}{1}".format(month, year))
wdpa_status_yr_2022_and_less_path = os.path.join(wdpa_output_path,"wdpa_poly_point_public_restricted_basemap_intersect_statusyr_2022_and_less_{0}{1}".format(month, year))
wdpa_status_yr_2024_and_less_path = os.path.join(wdpa_output_path,"wdpa_poly_point_public_restricted_basemap_intersect_statusyr_2024_and_less_{0}{1}".format(month, year))
wdoecm_status_yr_2020_and_less_path = os.path.join(wdoecm_output_path,"wdoecm_poly_point_basemap_intersect_statusyr_2020_and_less_{0}{1}".format(month, year))
wdoecm_status_yr_2022_and_less_path = os.path.join(wdoecm_output_path,"wdoecm_poly_point_basemap_intersect_statusyr_2022_and_less_{0}{1}".format(month, year))
wdoecm_status_yr_2024_and_less_path = os.path.join(wdoecm_output_path,"wdoecm_poly_point_basemap_intersect_statusyr_2024_and_less_{0}{1}".format(month, year))

#Subset pre-processed WDPA and WDOECM by STATUS_YR
arcpy.conversion.ExportFeatures(wdpa_public_restricted_basemap_intersect_output_path, wdpa_status_yr_2020_and_less_path, "STATUS_YR <= 2020")
arcpy.conversion.ExportFeatures(wdpa_public_restricted_basemap_intersect_output_path, wdpa_status_yr_2022_and_less_path, "STATUS_YR <= 2022")
arcpy.conversion.ExportFeatures(wdpa_public_restricted_basemap_intersect_output_path, wdpa_status_yr_2024_and_less_path, "STATUS_YR <= 2024")
arcpy.conversion.ExportFeatures(wdoecm_basemap_intersect_output_path, wdoecm_status_yr_2020_and_less_path, "STATUS_YR <= 2020")
arcpy.conversion.ExportFeatures(wdoecm_basemap_intersect_output_path, wdoecm_status_yr_2022_and_less_path, "STATUS_YR <= 2022")
arcpy.conversion.ExportFeatures(wdoecm_basemap_intersect_output_path, wdoecm_status_yr_2024_and_less_path, "STATUS_YR <= 2024")

#Define fields to flatten (pairwise dissolve) WDPA and WDOECM (subset by different STATUS_YR values)
## PA_DEF = WDPA/WDOECM field, binary value (0 = OECM, 1 = protected area)
## Sub_region_NAME = Name of UNSD M49 sub-region in global basemap
## ISO3 = WDPA/WDOECM field, containing International Standards Organization (ISO) 3 digit code corresponding to a country or territory in the WDPA/WDOECM.

flattenGroups = ["PA_DEF", "Sub_region_Name", "ISO3"]

#Create flattened WDPA for STATUS_YR <= 2024 for each field name in flattenGroups
for i in flattenGroups:
    output_path = f"{wdpa_status_yr_2024_and_less_path}_{i}"
    arcpy.analysis.PairwiseDissolve(wdpa_status_yr_2024_and_less_path, output_path, i,None, "SINGLE_PART")
    arcpy.management.RepairGeometry(output_path, "DELETE_NULL", "OGC") 

#Create flattened WDPA for STATUS_YR <= 2022 for each field name in flattenGroups
for i in flattenGroups:
    output_path = f"{wdpa_status_yr_2022_and_less_path}_{i}"
    arcpy.analysis.PairwiseDissolve(wdpa_status_yr_2022_and_less_path, output_path, i,None, "SINGLE_PART")
    arcpy.management.RepairGeometry(output_path, "DELETE_NULL", "OGC")

#Create flattened WDPA for STATUS_YR <= 2020 for each field name in flattenGroups
for i in flattenGroups:
    output_path = f"{wdpa_status_yr_2020_and_less_path}_{i}"
    arcpy.analysis.PairwiseDissolve(wdpa_status_yr_2020_and_less_path, output_path, i,None, "SINGLE_PART")
    arcpy.management.RepairGeometry(output_path, "DELETE_NULL", "OGC")

#Create flattened WDOECM for STATUS_YR <= 2024 for each field name in flattenGroups    
for i in flattenGroups:
    output_path = f"{wdoecm_status_yr_2024_and_less_path}_{i}"
    arcpy.analysis.PairwiseDissolve(wdoecm_status_yr_2024_and_less_path, output_path, i,None, "SINGLE_PART")
    arcpy.management.RepairGeometry(output_path, "DELETE_NULL", "OGC")
    
#Create flattened WDOECM for STATUS_YR <= 2022 for each field name in flattenGroups
for i in flattenGroups:
    output_path = f"{wdoecm_status_yr_2022_and_less_path}_{i}"
    arcpy.analysis.PairwiseDissolve(wdoecm_status_yr_2022_and_less_path, output_path, i,None, "SINGLE_PART")
    arcpy.management.RepairGeometry(output_path, "DELETE_NULL", "OGC")

#Create flattened WDOECM for STATUS_YR <= 2020 for each field name in flattenGroups
for i in flattenGroups:
    output_path = f"{wdoecm_status_yr_2020_and_less_path}_{i}"
    arcpy.analysis.PairwiseDissolve(wdoecm_status_yr_2020_and_less_path, output_path, i,None, "SINGLE_PART")
    arcpy.management.RepairGeometry(output_path, "DELETE_NULL", "OGC")

#### Buffer Points 